In [24]:
data_dir = "/home/xutingfeng/GIFT/"  # 数据存放路径
gwas_pheno = "/home/xutingfeng/ukb/ukbData/phenotypes/kidney"
gwas_file = data_dir + "data/GWAS/T1Mapping_Cortex_20240129.csv_firstorder_Median_all_2023_GRCh38_unionKidneys.tsv.gz"
# pfile = "/home/xutingfeng/ukb/externel/1kg/g1000_eur_GRCh38"
pfile = "/home/xutingfeng/GIFT/data/pgen/DNAJC16_GRCh38_union"
pheno_path = "/home/xutingfeng/ukb/ukbData/phenotypes/kidney/union_v3_INT.tsv"
cov_path = "/home/xutingfeng/ukb/ukbData/phenotypes/tf/all_pheno.pkl"

In [25]:
import pandas as pd 
from finemap_tools.reression import cal_residual

In [26]:
from pgenlib_tools import PgenReaderFull

cov = pd.read_pickle(cov_path)
pheno = pd.read_csv(pheno_path, sep="\s+").set_index("FID")[ 'T1Mapping_Cortex_20240129.csv_firstorder_Median']
geno = PgenReaderFull(pfile_path=pfile)




In [27]:
all_phenos = cov.merge(pheno, left_index=True, right_index=True)


In [28]:
all_phenos.columns

Index(['sex', 'height', 'weight', 'BSA', 'genotype_array', 'age', 'PC1', 'PC2',
       'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'assessment_center', 'age_squared', 'BMI', 'dbp_a', 'sbp_a', 'tc_a',
       'tg_a', 'hdl_a', 'ldl_a', 'dbp', 'sbp', 'tc', 'tg', 'hdl', 'ldl',
       'apob', 'cad', 'mi', 'crp', 'eGFR', 'Cr', 'CAD', 'cad_date',
       'birth_date', 'recuit_date', 'death_date', 'cad_age', 'death_age',
       'recuit_age', 'year_of_cad_after_recuit', 'survival_time',
       'is_earily_CAD', 'incident_cad', 'cad_status', 'prevalent_cad',
       'T1Mapping_Cortex_20240129.csv_firstorder_Median'],
      dtype='object')

In [29]:
cofounder = ["age", "sex", "genotype_array", "BMI",*[f"PC{i}" for i in range(1, 11)]]
y = "T1Mapping_Cortex_20240129.csv_firstorder_Median"


In [31]:
# with open("/home/xutingfeng/GIFT/data/people.tsv", "wb") as f: 
#     for i in all_phenos.index: 
#         f.write(f"{i}\t{i}\n".encode())

In [32]:
all_phenos.dropna(subset=["T1Mapping_Cortex_20240129.csv_firstorder_Median", "age", "sex", "genotype_array", "BMI",*[f"PC{i}" for i in range(1, 11)]], inplace=True)

In [33]:
for col in cofounder + [y]:
    all_phenos[col] = all_phenos[col].astype(float)
    # all_phenos[col] = cal_residual(all_phenos, col, y)

In [16]:
all_phenos = cal_residual(all_phenos, cofounder, y, plus_mean=False, )


passed data have 4891 rows
used data have 4891 rows after dropna


In [34]:
from pathlib import Path
from finemap_tools.utils import add_ID
from finemap_tools.reader.gwas import load_GWASFormated_file
topLoci = (1, 15583355, 15583355) # (chr, start, end)
locus_range = 1000 # 100kb 
locus_range = locus_range * 1000 
locus_range_tuple = (topLoci[0], topLoci[1] - locus_range, topLoci[2] + locus_range)

locus_region = f"{locus_range_tuple[0]}:{locus_range_tuple[1]}-{locus_range_tuple[2]}"
print(f"locus_region: {locus_region}")


gwas = load_GWASFormated_file(file_path = gwas_file, region = locus_region )
gwas['ID'] = add_ID(gwas, col_list=["chrom", "pos", "ref", "alt"])

# 1. filter out 
from finemap_tools.snpfilter.snp import filter_pipline
clean = filter_pipline(gwas['ID'], id_sep=':')
gwas = gwas[gwas['ID'].isin(clean)]
gwas 

locus_region: 1:14583355-16583355
tabix have a header, so will take the first line as header and remove it.
drop 2021 ambiguous alleles
drop 42 biallelic snps


,chrom,pos,ref,alt,pval,af,beta,sebeta,ID
1,1,14583587,G,A,0.8260777028074947,0.15877,0.00586118,0.0266739,1:14583587:G:A
2,1,14583907,T,C,0.13119884144284497,0.000677201,-0.556895,0.368953,1:14583907:T:C
3,1,14583992,G,A,0.29227387666099736,0.000677507,0.388817,0.369195,1:14583992:G:A
4,1,14584191,G,A,0.1256666390369755,0.0012435,-0.423208,0.276351,1:14584191:G:A
5,1,14584347,C,T,0.161866881591971,0.00544341,0.183348,0.131073,1:14584347:C:T
...,...,...,...,...,...,...,...,...,...
16202,1,16582952,G,T,0.15836528985352105,0.0600916,0.0744191,0.0527574,1:16582952:G:T
16203,1,16583062,G,A,0.30269691864207554,0.000915122,0.331578,0.321712,1:16583062:G:A
16204,1,16583105,A,C,0.9861350837938949,0.0113412,-0.00165553,0.095266,1:16583105:A:C
16205,1,16583157,G,A,0.5919854055206714,0.00247408,-0.106121,0.198001,1:16583157:G:A


In [35]:
used = set(geno.var_list)&set(gwas['ID'])

print(f"used: {len(used)}")

used: 14075


In [36]:
sample_eid = [i for  i in geno.get_sample_idx_by_IID(all_phenos.index.tolist()) if i != None]
sample_eid


2563517 not found in IID_list
4105312 not found in IID_list
4194469 not found in IID_list
4599150 not found in IID_list
4889734 not found in IID_list
5379659 not found in IID_list
5832872 not found in IID_list


[902,
 4012,
 3839,
 2320,
 2349,
 3287,
 4098,
 1771,
 298,
 955,
 294,
 2278,
 3356,
 2108,
 192,
 1602,
 1297,
 1108,
 1063,
 4659,
 4272,
 4162,
 1736,
 2252,
 2575,
 171,
 1190,
 3987,
 2640,
 3233,
 1242,
 382,
 3079,
 3037,
 2673,
 1712,
 3201,
 611,
 372,
 4636,
 3681,
 3074,
 1912,
 4552,
 823,
 631,
 4576,
 736,
 1305,
 692,
 4547,
 2598,
 4462,
 907,
 4883,
 3585,
 1651,
 1925,
 2264,
 4410,
 765,
 3139,
 82,
 4531,
 622,
 1601,
 2133,
 1637,
 99,
 2731,
 3988,
 1485,
 937,
 3497,
 3368,
 445,
 1501,
 1563,
 2506,
 2482,
 2296,
 610,
 1965,
 2422,
 1738,
 1579,
 377,
 3550,
 4479,
 3431,
 4670,
 1586,
 1505,
 3303,
 3267,
 2574,
 3040,
 2653,
 4401,
 3008,
 1975,
 3689,
 1918,
 2853,
 2122,
 2489,
 122,
 4844,
 882,
 3586,
 2038,
 992,
 654,
 2556,
 3757,
 719,
 1377,
 3122,
 579,
 3085,
 3674,
 4113,
 855,
 4369,
 3273,
 4402,
 2848,
 1374,
 1704,
 4470,
 1257,
 812,
 325,
 3088,
 3554,
 3249,
 2973,
 3459,
 3257,
 520,
 827,
 1706,
 3103,
 3050,
 3481,
 1769,
 1861,
 3771,

In [37]:
geno_array = geno.extract(variant_ids=list(used), sample_idx=sample_eid, asFrame=True)


In [38]:
all_phenos = all_phenos.loc[geno_array.index]


In [39]:
# for col in geno_array.columns:
#     geno_array[col] = (geno_array[col].values - geno_array[col].mean())/geno_array[col].std()

In [30]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler_X = scaler.fit(all_phenos)
# scaler_X

In [42]:
import seaborn as sns 
# sns.kdeplot(geno_array.isna().sum(axis=0).sort_values()/ geno_array.shape[0])
((geno_array.isna().sum(axis=0).sort_values()/ geno_array.shape[0] ) == 0).sum()

1393

In [31]:
all_phenos.reset_index(drop=False).to_feather("/home/xutingfeng/GIFT/DNAJC16_Y.feather")
geno_array.reset_index(drop=True).to_feather("/home/xutingfeng/GIFT/DNAJC16_X.feather")
R.

In [2]:
geno_array

NameError: name 'geno_array' is not defined